# BAP for showing summary information for an area from OBIS

This is a potential new Biogeographic Analysis Package (for the National Biogeographic Map and other uses) that would respond to a selection of one of the geographic areas of interest summarized in in the Ocean Biogeographic Information System. Summary information is pretty simple, and I'll create some other notebooks that pull back more interesting information on species observations in the area.

The geographic areas include EEZ zones, Ecologically and Biologically Significant Areas (EBSAs), and Marine Protected Areas. The data for these was originally pulled from the MarineRegions.org aggregation point with some work to make the geospatial data actually work. Eventually, all of this needs to be in the Spatial Feature Registry and rectified with other sources like PAD-US. For now, I used a service provided by Pieter Provoost to pull down data.

http://iobis.org/geoserver/OBIS/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=OBIS:areas&maxFeatures=9999&outputFormat=SHAPE-ZIP&format_options=CHARSET:UTF-8

Note: I had to add the UTF-8 option after discovering that the text properties coming from this WFS (via Geoserver and PostGIS, I believe) were in a different encoding.

I used GDAL (ogr2ogr) to extract just the properties I wanted for our purposes (the WFS contains high level point in time summaries for the areas that we are wanting to pull in real time from another service).

`ogr2ogr -f GeoJSON -t_srs crs:84 obisareas.geojson areasPolygon.shp -sql "SELECT gid AS areaid, name, country, type FROM areasPolygon"`

I did this to set the data up for import into the GC2 platform we are experimenting with, changing the gid to areaid from the original data since I needed to import this into another PostgreSQL database in GC2 (and it was horking on trying to deal with an existing gid property).

The rest of the notebook shows a little bit of how we might interact with the geographic areas. We may or may not want to limit the available areas to those for the United States.

In [28]:
import requests
from IPython.display import display,HTML

This next section goes out and gets data on specific areas from the Elasticsearch end point on our nascent first try at hosting the GC2 infrastructure on the cloud (we'll have to replace this because it is very transient). I pulled just the properties for now. You can execute the same query but leave off the fields:[] part to grab up the geometry. I'll play with that as well. In an NBM rendering of this BAP, we would want to show the area highlighted on the map.

This query gets just US areas, but we would probably want to just include everything in our NBM - no reason not to unless we run into sensitivities about it being our "National" Biogeographic Map.

In [32]:
# Something like this would get US areas for lookup
# This goes against the test GC2 instance using the Elasticsearch API and brings back just the properties.
# Still need to do a bunch of stuff to set up the Spatial Feature Registry for real.
usAreas = requests.get('http://52.90.54.105/api/v1/elasticsearch/search/bcb/public/obisareas?q={%22fields%22:[%22properties.name%22,%22properties.country%22,%22properties.areaid%22,%22properties.type%22],%22query%22:{%22constant_score%22:{%22filter%22:{%22term%22:{%22country%22:%22United%20States%22}}}}}&size=100').json()
display(usAreas)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '117',
    '_index': 'bcb_public_obisareas',
    '_score': 1.0,
    '_type': 'obisareas',
    'fields': {'properties.areaid': [249],
     'properties.country': ['United States'],
     'properties.name': ['United States: Puerto Rico and Virgin Islands of the United States'],
     'properties.type': ['eez']}},
   {'_id': '179',
    '_index': 'bcb_public_obisareas',
    '_score': 1.0,
    '_type': 'obisareas',
    'fields': {'properties.areaid': [250],
     'properties.country': ['United States'],
     'properties.name': ['United States: Wake Island'],
     'properties.type': ['eez']}},
   {'_id': '264',
    '_index': 'bcb_public_obisareas',
    '_score': 1.0,
    '_type': 'obisareas',
    'fields': {'properties.areaid': [247],
     'properties.country': ['United States'],
     'properties.name': ['United States: Northern Mariana Islands and Guam'],
     'properties.type': ['eez']}},
   {'_id': '428',
    '

This next section loops through the returned areas and calls up the simplified backend service from iOBIS. The link on the area name at the top of each result goes to the iOBIS portal page for that area. These data are shown there at the top of those pages in the Overview section, and that's basically what we would want to show for this simple BAP. I'll think about whether or not we want to provide the direct download links to pull data. We will want to provide a link in our rendering of this information back to the iOBIS pages.

In [33]:
# Loop the US areas to make links to get the backend OBIS data summarizing the areas
for hit in usAreas["hits"]["hits"]:
    areaId = str(hit["fields"]["properties.areaid"][0])
    display(HTML("<a target='_new' href='http://www.iobis.org/explore/#/area/"+areaId+"'>"+hit["fields"]["properties.name"][0]+"</a>"))
    areaSummary = requests.get("http://backend.iobis.org/area/"+areaId).json()
    display(areaSummary)

{'endemic': 229,
 'gid': 249,
 'gisd': 10,
 'hab': 5,
 'institutes': 57,
 'maxlat': 21.8542523682177,
 'maxlon': -63.88888793414,
 'minlat': 14.9281934301631,
 'minlon': -68.4818895564666,
 'name': 'United States: Puerto Rico and Virgin Islands of the United States',
 'records': 163090,
 'redlist': 476,
 'resources': 71,
 'species': 2222,
 'surface': 0.0,
 'taxa': 3553,
 'type': 'eez',
 'url': None}

{'endemic': 6,
 'gid': 250,
 'gisd': 0,
 'hab': 0,
 'institutes': 20,
 'maxlat': 22.6517483672172,
 'maxlon': 170.187168656926,
 'minlat': 16.0373143741692,
 'minlon': 163.071049652078,
 'name': 'United States: Wake Island',
 'records': 16682,
 'redlist': 150,
 'resources': 23,
 'species': 316,
 'surface': 0.0,
 'taxa': 422,
 'type': 'eez',
 'url': None}

{'endemic': 68,
 'gid': 247,
 'gisd': 5,
 'hab': 0,
 'institutes': 45,
 'maxlat': 23.8930565599986,
 'maxlon': 149.544456639993,
 'minlat': 10.9508323601797,
 'minlon': 141.19510163,
 'name': 'United States: Northern Mariana Islands and Guam',
 'records': 189783,
 'redlist': 441,
 'resources': 51,
 'species': 1577,
 'surface': 0.0,
 'taxa': 2848,
 'type': 'eez',
 'url': None}

{'endemic': 20,
 'gid': 240,
 'gisd': 4,
 'hab': 0,
 'institutes': 24,
 'maxlat': -10.0238876636186,
 'maxlon': -165.200832650869,
 'minlat': -17.5577786538969,
 'minlon': -173.746667652248,
 'name': 'United States: American Samoa',
 'records': 163911,
 'redlist': 421,
 'resources': 30,
 'species': 869,
 'surface': 0.0,
 'taxa': 1171,
 'type': 'eez',
 'url': None}

{'endemic': 625,
 'gid': 241,
 'gisd': 33,
 'hab': 0,
 'institutes': 58,
 'maxlat': 31.797866959999,
 'maxlon': 180.0000005,
 'minlat': 15.5631229400004,
 'minlon': -180.00000083,
 'name': 'United States: Hawaii',
 'records': 505336,
 'redlist': 607,
 'resources': 92,
 'species': 3106,
 'surface': 0.0,
 'taxa': 5084,
 'type': 'eez',
 'url': None}

...

{'endemic': 2,
 'gid': 248,
 'gisd': 2,
 'hab': 0,
 'institutes': 16,
 'maxlat': 9.78681235999055,
 'maxlon': -159.374721661511,
 'minlat': 2.66388835102225,
 'minlon': -165.755850649998,
 'name': 'United States: Palmyra Atoll',
 'records': 93915,
 'redlist': 337,
 'resources': 24,
 'species': 645,
 'surface': 0.0,
 'taxa': 866,
 'type': 'eez',
 'url': None}

{'endemic': 1497,
 'gid': 246,
 'gisd': 45,
 'hab': 26,
 'institutes': 106,
 'maxlat': 49.150001,
 'maxlon': -116.915861261173,
 'minlat': 30.5419996904644,
 'minlon': -129.16350658,
 'name': 'United States: North Pacific',
 'records': 1351829,
 'redlist': 512,
 'resources': 170,
 'species': 5463,
 'surface': 0.0,
 'taxa': 8651,
 'type': 'eez',
 'url': None}

{'endemic': 9,
 'gid': 244,
 'gisd': 1,
 'hab': 0,
 'institutes': 27,
 'maxlat': 20.13391537,
 'maxlon': -165.97304365004,
 'minlat': 13.3587963600356,
 'minlon': -173.034017659959,
 'name': 'United States: Johnston Atoll',
 'records': 25312,
 'redlist': 124,
 'resources': 34,
 'species': 392,
 'surface': 0.0,
 'taxa': 690,
 'type': 'eez',
 'url': None}

{'endemic': 68,
 'gid': 238,
 'gisd': 1,
 'hab': 18,
 'institutes': 39,
 'maxlat': 74.709767999999,
 'maxlon': -137.568887900557,
 'minlat': 65.7755048425741,
 'minlon': -168.97629488,
 'name': 'United States: Alaska: Arctic Ocean',
 'records': 76887,
 'redlist': 64,
 'resources': 60,
 'species': 971,
 'surface': 0.0,
 'taxa': 1941,
 'type': 'eez',
 'url': None}

{'endemic': 3623,
 'gid': 245,
 'gisd': 109,
 'hab': 51,
 'institutes': 141,
 'maxlat': 45.165001,
 'maxlon': -65.6997212200607,
 'minlat': 23.817499,
 'minlon': -97.969138722761,
 'name': 'United States: North Atlantic',
 'records': 5329697,
 'redlist': 1675,
 'resources': 430,
 'species': 17667,
 'surface': 0.0,
 'taxa': 25647,
 'type': 'eez',
 'url': None}